# Data science in Microsoft Fabric

In [2]:
# Azure storage access info for open dataset diabetes
blob_account_name = "azureopendatastorage"
blob_container_name = "mlsamples"
blob_relative_path = "diabetes"
blob_sas_token = r"" # Blank since container is Anonymous access

# Set Spark config to access blob storage
wasbs_path = f"wasbs://%s@%s.blob.core.windows.net/%s" % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set("fs.azure.sas.%s.%s.blob.core.windows.net" % (blob_container_name, blob_account_name), blob_sas_token)
print("Remote blob path: " + wasbs_path)

# Spark read parquet
df = spark.read.parquet(wasbs_path)

StatementMeta(, 12ec0c47-5f63-4537-aa87-41f8f33a5dd0, 4, Finished, Available, Finished)

Remote blob path: wasbs://mlsamples@azureopendatastorage.blob.core.windows.net/diabetes


In [2]:
display(df)

StatementMeta(, 3406d6f7-16dc-499c-a564-bfa8b7fabea7, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8840ce15-342e-4319-a97e-07eca0b3db5a)

In [3]:
df = df.toPandas()
df.head()

StatementMeta(, 12ec0c47-5f63-4537-aa87-41f8f33a5dd0, 5, Finished, Available, Finished)

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,59,2,32.1,101.0,157,93.2,38.0,4.0,4.8598,87,151
1,48,1,21.6,87.0,183,103.2,70.0,3.0,3.8918,69,75
2,72,2,30.5,93.0,156,93.6,41.0,4.0,4.6728,85,141
3,24,1,25.3,84.0,198,131.4,40.0,5.0,4.8903,89,206
4,50,1,23.0,101.0,192,125.4,52.0,4.0,4.2905,80,135


In [4]:
def clean_data(df):
    # Created column 'Risk' from formula
    # Check if the value in column 'Y' is greater than 211.5, 
    # if yes, assign 1 otherwise 0, and create a new column 'Risk' with these values.
    df['Risk'] = (df['Y'] > 211.5).astype(int)
    
    # Return the updated DataFrame
    return df

# Make a copy of the DataFrame 'df'
df_clean = clean_data(df.copy())

# Display the cleaned DataFrame
display(df_clean)

StatementMeta(, 12ec0c47-5f63-4537-aa87-41f8f33a5dd0, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4fc414bc-2f51-41cd-817a-a836f795067d)

In [5]:
df_clean.describe()

StatementMeta(, 12ec0c47-5f63-4537-aa87-41f8f33a5dd0, 7, Finished, Available, Finished)

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y,Risk
count,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000
mean,48.518100,1.468326,26.375792,94.647014,189.140271,115.439140,49.788462,4.070249,4.641411,91.260181,152.133484,0.251131
std,13.109028,0.499561,4.418122,13.831283,34.608052,30.413081,12.934202,1.290450,0.522391,11.496335,77.093005,0.434155
min,19.000000,1.000000,18.000000,62.000000,97.000000,41.600000,22.000000,2.000000,3.258100,58.000000,25.000000,0.000000
25%,38.250000,1.000000,23.200000,84.000000,164.250000,96.050000,40.250000,3.000000,4.276700,83.250000,87.000000,0.000000
50%,50.000000,1.000000,25.700000,93.000000,186.000000,113.000000,48.000000,4.000000,4.620050,91.000000,140.500000,0.000000
75%,59.000000,2.000000,29.275000,105.000000,209.750000,134.500000,57.750000,5.000000,4.997200,98.000000,211.500000,0.750000
max,79.000000,2.000000,42.200000,133.000000,301.000000,242.400000,99.000000,9.090000,6.107000,124.000000,346.000000,1.000000


In [6]:
from sklearn.model_selection import train_test_split
    
X, y = df_clean[['AGE','SEX','BMI','BP','S1','S2','S3','S4','S5','S6']].values, df_clean['Y'].values
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

StatementMeta(, 12ec0c47-5f63-4537-aa87-41f8f33a5dd0, 8, Finished, Available, Finished)

In [7]:
import mlflow
experiment_name = "shsakam-diabetes-regression"
mlflow.set_experiment(experiment_name)

StatementMeta(, 12ec0c47-5f63-4537-aa87-41f8f33a5dd0, 9, Finished, Available, Finished)

2025/04/29 03:46:57 INFO mlflow.tracking.fluent: Experiment with name 'shsakam-diabetes-regression' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1745898421299, experiment_id='c8e0872f-8429-46de-a4e0-3af487c4ddf6', last_update_time=None, lifecycle_stage='active', name='shsakam-diabetes-regression', tags={}>

In [8]:
from sklearn.linear_model import LinearRegression
    
with mlflow.start_run():
   mlflow.autolog()
    
   model = LinearRegression()
   model.fit(X_train, y_train)

StatementMeta(, 12ec0c47-5f63-4537-aa87-41f8f33a5dd0, 10, Finished, Available, Finished)

2025/04/29 03:48:11 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/04/29 03:48:35 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [9]:
from sklearn.model_selection import train_test_split
    
X, y = df_clean[['AGE','SEX','BMI','BP','S1','S2','S3','S4','S5','S6']].values, df_clean['Risk'].values
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

StatementMeta(, 12ec0c47-5f63-4537-aa87-41f8f33a5dd0, 11, Finished, Available, Finished)

In [10]:
import mlflow
experiment_name = "shsakam-diabetes-classification"
mlflow.set_experiment(experiment_name)


StatementMeta(, 12ec0c47-5f63-4537-aa87-41f8f33a5dd0, 12, Finished, Available, Finished)

2025/04/29 03:50:36 INFO mlflow.tracking.fluent: Experiment with name 'shsakam-diabetes-classification' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1745898636748, experiment_id='b67542fd-2a2f-4ccb-8f51-a7690906d936', last_update_time=None, lifecycle_stage='active', name='shsakam-diabetes-classification', tags={}>

In [11]:
from sklearn.linear_model import LogisticRegression
    
with mlflow.start_run():
    mlflow.sklearn.autolog()

    model = LogisticRegression(C=1/0.1, solver="liblinear").fit(X_train, y_train)

StatementMeta(, 12ec0c47-5f63-4537-aa87-41f8f33a5dd0, 13, Finished, Available, Finished)

In [15]:
from sklearn.tree import DecisionTreeRegressor
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec

mlflow.set_experiment(experiment_name)
with mlflow.start_run():
    mlflow.autolog(log_models=False)
    model = DecisionTreeRegressor(max_depth=5)
    model.fit(X_train, y_train)
       
    # Define the model signature
    input_schema = Schema([
        ColSpec("integer", "AGE"),
        ColSpec("integer", "SEX"),\
        ColSpec("double", "BMI"),
        ColSpec("double", "BP"),
        ColSpec("integer", "S1"),
        ColSpec("double", "S2"),
        ColSpec("double", "S3"),
        ColSpec("double", "S4"),
        ColSpec("double", "S5"),
        ColSpec("integer", "S6"),
     ])
    output_schema = Schema([ColSpec("integer")])
    signature = ModelSignature(inputs=input_schema, outputs=output_schema)
   
    # Log the model
    mlflow.sklearn.log_model(model, "model", signature=signature)

StatementMeta(, 12ec0c47-5f63-4537-aa87-41f8f33a5dd0, 17, Finished, Available, Finished)

In [17]:
# Get the most recent experiement run
exp = mlflow.get_experiment_by_name(experiment_name)
last_run = mlflow.search_runs(exp.experiment_id, order_by=["start_time DESC"], max_results=1)
last_run_id = last_run.iloc[0]["run_id"]

# Register the model that was trained in that run
print("Registering the model from run :", last_run_id)
model_uri = "runs:/{}/model".format(last_run_id)
mv = mlflow.register_model(model_uri, "shsakam-diabetes-model")
print("Name: {}".format(mv.name))
print("Version: {}".format(mv.version))

StatementMeta(, 12ec0c47-5f63-4537-aa87-41f8f33a5dd0, 19, Finished, Available, Finished)

Registering the model from run : 10551fd3-58f8-4385-92de-5864f3e5ef4f


Successfully registered model 'shsakam-diabetes-model'.
Created version '1' of model 'shsakam-diabetes-model'.


In [18]:
from pyspark.sql.types import IntegerType, DoubleType

# Create a new dataframe with patient data
data = [
    (62, 2, 33.7, 101.0, 157, 93.2, 38.0, 4.0, 4.8598, 87),
    (50, 1, 22.7, 87.0, 183, 103.2, 70.0, 3.0, 3.8918, 69),
    (76, 2, 32.0, 93.0, 156, 93.6, 41.0, 4.0, 4.6728, 85),
    (25, 1, 26.6, 84.0, 198, 131.4, 40.0, 5.0, 4.8903, 89),
    (53, 1, 23.0, 101.0, 192, 125.4, 52.0, 4.0, 4.2905, 80),
    (24, 1, 23.7, 89.0, 139, 64.8, 61.0, 2.0, 4.1897, 68),
    (38, 2, 22.0, 90.0, 160, 99.6, 50.0, 3.0, 3.9512, 82),
    (69, 2, 27.5, 114.0, 255, 185.0, 56.0, 5.0, 4.2485, 92),
    (63, 2, 33.7, 83.0, 179, 119.4, 42.0, 4.0, 4.4773, 94),
    (30, 1, 30.0, 85.0, 180, 93.4, 43.0, 4.0, 5.3845, 88)
]
columns = ['AGE','SEX','BMI','BP','S1','S2','S3','S4','S5','S6']
df = spark.createDataFrame(data, schema=columns)

# Convert data types to match the model input schema
df = df.withColumn("AGE", df["AGE"].cast(IntegerType()))
df = df.withColumn("SEX", df["SEX"].cast(IntegerType()))
df = df.withColumn("BMI", df["BMI"].cast(DoubleType()))
df = df.withColumn("BP", df["BP"].cast(DoubleType()))
df = df.withColumn("S1", df["S1"].cast(IntegerType()))
df = df.withColumn("S2", df["S2"].cast(DoubleType()))
df = df.withColumn("S3", df["S3"].cast(DoubleType()))
df = df.withColumn("S4", df["S4"].cast(DoubleType()))
df = df.withColumn("S5", df["S5"].cast(DoubleType()))
df = df.withColumn("S6", df["S6"].cast(IntegerType()))

# Save the data in a delta table
table_name = "diabetes_test"
df.write.format("delta").mode("overwrite").saveAsTable(table_name)
print(f"Spark dataframe saved to delta table: {table_name}")

StatementMeta(, 12ec0c47-5f63-4537-aa87-41f8f33a5dd0, 20, Finished, Available, Finished)

Spark dataframe saved to delta table: diabetes_test


In [20]:
import mlflow
from synapse.ml.predict import MLFlowTransformer

## Read the patient features data 
df_test = spark.read.format("delta").load(f"Tables/{table_name}")

# Use the model to generate diabetes predictions for each row
model = MLFlowTransformer(
    inputCols=["AGE","SEX","BMI","BP","S1","S2","S3","S4","S5","S6"],
    outputCol="predictions",
    modelName="shsakam-diabetes-model",
    modelVersion=1)
df_test = model.transform(df)

# Save the results (the original features PLUS the prediction)
df_test.write.format('delta').mode("overwrite").option("mergeSchema", "true").saveAsTable(table_name)

StatementMeta(, 12ec0c47-5f63-4537-aa87-41f8f33a5dd0, 22, Finished, Available, Finished)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.


StatementMeta(, 12ec0c47-5f63-4537-aa87-41f8f33a5dd0, 23, Finished, Available, Finished)